# Prerequisites

In [ ]:
from tm.optimizers import GeneticOptimizer, calculate_absolute_buy_and_hold_returns, StrategyPerformanceEvaluator
from tm.trading_rules import SimpleMovingAverage, ExponentialMovingAverage, STO, MACD, RSI, ROC
from tm import StockDataProvider
from tm.optimizers.utils import map_chromosome_to_trading_rule_parameters, filter_for_active_rules
from tm.backtesting import GeometricBrownianMotion, MonteCarloCrossValidation
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
trading_rules = [SimpleMovingAverage, ExponentialMovingAverage, STO, MACD, RSI, ROC]

# Optimization with Genetic Algorithm

In [ ]:
data = StockDataProvider('CSCO', start='2014-11-01', end='2015-03-31')
optimizer = GeneticOptimizer(data, trading_rules)

In [ ]:
%%time
# Returns hall of fame of best hof_size individuals
hof = optimizer.run(pop_size=1000, ngen=5, hof_size=100)

In [ ]:
for individual in hof:
    print('Chromosome:', individual)
    print('Parameter mapping:', map_chromosome_to_trading_rule_parameters(individual, trading_rules))
    print('Net profit for time period:', optimizer.toolbox.evaluate(individual))
    print('----------------------------------------')

In [ ]:
# Plot buy and sell signals of best strategy
best_strategy = hof[0]
rule_instances = list(map(lambda Rule, params: Rule(data, *params), trading_rules, map_chromosome_to_trading_rule_parameters(best_strategy, trading_rules)))
active_rule_instances = filter_for_active_rules(best_strategy, rule_instances)
evaluator = StrategyPerformanceEvaluator(active_rule_instances)
print('Net profit:', evaluator.calculate_net_profit())
buy_signals = evaluator.buy_signals
sell_signals = evaluator.sell_signals
print('Num of sell signals:', len(sell_signals[sell_signals == True]))
print('Num of buy signals:', len(buy_signals[buy_signals == True]))

In [ ]:
# Performance of buy and hold
calculate_absolute_buy_and_hold_returns(data)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))
ax.plot(data.history['Close'], label='Stock prices')

buy_points = pd.Series(data=map(lambda index, price: price if buy_signals.loc[index] == True else np.nan, data.history['Close'].index, data.history['Close']),
                       index=data.history['Close'].index)
ax.scatter(x=buy_points.index, y=buy_points, color='green', marker='^', label='Buy signal')
sell_points = pd.Series(data=map(lambda index, price: price if sell_signals.loc[index] == True else np.nan, data.history['Close'].index, data.history['Close']),
                       index=data.history['Close'].index)
ax.scatter(x=sell_points.index, y=sell_points, color='red', marker='v', label='Sell signal')

ax.set_xlabel('Date')
ax.set_ylabel('Stock price')
ax.legend()
fig.tight_layout()